In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


from tensorflow.keras.layers import Input, Dense, concatenate, LSTM, Bidirectional, Dropout, GRU
from tensorflow.keras.models import Model
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)


In [2]:
df_static = pd.read_csv('../treated_data/df_static.csv',index_col=0)
df_dynamic = pd.read_csv('../treated_data/df_dynamic.csv', index_col=0)

/home/mateus/anaconda3/envs/home-credit/lib/python3.8/site-packages/numpy/lib/arraysetops.py:580: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
X_train, X_test, y_train, y_test  = train_test_split(df_static.index ,df_static.TARGET, test_size=0.33, random_state=42, shuffle=False)

In [4]:
X_static_train = df_static.loc[X_train]
X_static_train = X_static_train.fillna(X_static_train.mean()).values[:,1:]

X_static_test = df_static.loc[X_test]
X_static_test = X_static_test.fillna(X_static_train.mean()).values[:,1:]

In [5]:
X_dynamic_train = df_dynamic.loc[X_train].reset_index()
X_dynamic_train = X_dynamic_train.fillna(X_dynamic_train.mean())
X_dynamic_train = X_dynamic_train.pivot_table(values=list(set(X_dynamic_train.columns)-set(['SK_ID_PREV', 'MONTHS_BALANCE','SK_ID_CURR'])), index='SK_ID_CURR', columns='MONTHS_BALANCE',aggfunc='sum', fill_value=0)

In [6]:
X_dynamic_train = df_dynamic.loc[X_train].reset_index()
X_dynamic_train = X_dynamic_train.fillna(X_dynamic_train.mean())
X_dynamic_train = X_dynamic_train.pivot_table(values=list(set(X_dynamic_train.columns)-set(['SK_ID_PREV', 'MONTHS_BALANCE','SK_ID_CURR'])), index='SK_ID_CURR', columns='MONTHS_BALANCE',aggfunc='sum', fill_value=0).values


X_dynamic_test = df_dynamic.loc[X_test].reset_index()
X_dynamic_test = X_dynamic_test.fillna(X_dynamic_train.mean())
X_dynamic_test = X_dynamic_test.pivot_table(values=list(set(X_dynamic_test.columns)-set(['SK_ID_PREV', 'MONTHS_BALANCE','SK_ID_CURR'])), index='SK_ID_CURR', columns='MONTHS_BALANCE', aggfunc='sum', fill_value=0).values



In [7]:
X_dynamic_train_reshaped = np.zeros((33265, 97,57))

for l in range(33265):
    for o in range(97):
        for i in range(57):
            X_dynamic_train_reshaped[l,o,i]=X_dynamic_train[l,i*97+o]

            
X_dynamic_test_reshaped = np.zeros((16385, 97,57))

for l in range(16385):
    for o in range(97):
        for i in range(57):
            X_dynamic_test_reshaped[l,o,i]=X_dynamic_test[l,i*97+o]

In [8]:
scaler = StandardScaler()
X_static_train = scaler.fit_transform(X_static_train)
X_static_test = scaler.transform(X_static_test)

In [9]:
scalers = {}
for i in range(X_dynamic_train_reshaped.shape[2]):
    scalers[i] = MinMaxScaler()
    X_dynamic_train_reshaped[:, :, i] = scalers[i].fit_transform(X_dynamic_train_reshaped[:, :, i]) 

for i in range(X_dynamic_test_reshaped.shape[2]):
    X_dynamic_test_reshaped[:, :, 1] = scalers[i].transform(X_dynamic_test_reshaped[:, :, i]) 

In [10]:
# x_in_dy = Input(shape=(97,22))
# x_in_st = Input(shape=(200))

# x_dy = Bidirectional(LSTM(84, return_sequences=True))(x_in_dy)
# x_dy = Dropout(0.6)(x_dy)
# x_dy = Bidirectional(LSTM(46))(x_dy)

# x_st = Dense(100, activation="relu")(x_in_st)
# x_st = Dropout(0.6)(x_st)
# x_st = Dense(50, activation="relu")(x_st)



# z=concatenate([x_dy,x_st])

# out = Dense(24, activation='relu')(z)
# out = Dropout(0.5)(out)
# out = Dense(1, activation='sigmoid')(out)
# model1 = Model(inputs = [x_in_dy,x_in_st], outputs = out)


x_in_dy = Input(shape=(36,57))

x_dy = Bidirectional(LSTM(80, return_sequences=True))(x_in_dy)
x_dy = Dropout(0.5)(x_dy)
x_dy = Bidirectional(LSTM(80))(x_in_dy)

out = Dense(50, activation='relu')(x_dy)
# out = Dropout(0.5)(out)
out = Dense(1, activation='sigmoid')(out)
model1 = Model(inputs = x_in_dy, outputs = out)


In [11]:
# model1.compile(optimizer='rmsprop', 
#               loss = 'binary_crossentropy',
#               metrics=['accuracy'])
# history = model1.fit([X_dynamic_train, X_static_train], y_train, epochs=100, batch_size= 1000, validation_data=([X_dynamic_test, X_static_test], y_test)  )

In [12]:
opt = tf.keras.optimizers.SGD(lr=0.05)
m = tf.keras.metrics.AUC(num_thresholds=100)

In [13]:
# model1.compile(optimizer=opt, 
#               loss = 'binary_crossentropy',
#               metrics=[m])
# history = model1.fit( [X_dynamic_train_reshaped[:,-40:,:],X_static_train], y_train, epochs=500, batch_size= 500, validation_data=( [X_dynamic_test_reshaped[:,-40:,:],X_static_test], y_test)  )

model1.compile(optimizer='adam', 
              loss = 'binary_crossentropy',
              metrics=['accuracy'])
history = model1.fit( X_dynamic_train_reshaped[:,-36:,:], y_train, epochs=300, batch_size= 420, validation_data= (X_dynamic_test_reshaped[:,-36:,:], y_test))

Epoch 1/300
80/80 [==============================] - 4s 53ms/step - loss: 0.6936 - accuracy: 0.5050 - val_loss: 0.7023 - val_accuracy: 0.4966
Epoch 2/300
80/80 [==============================] - 4s 46ms/step - loss: 0.6927 - accuracy: 0.5105 - val_loss: 0.7012 - val_accuracy: 0.5046
Epoch 3/300
80/80 [==============================] - 4s 46ms/step - loss: 0.6921 - accuracy: 0.5138 - val_loss: 0.7095 - val_accuracy: 0.5101
Epoch 4/300
80/80 [==============================] - 4s 49ms/step - loss: 0.6917 - accuracy: 0.5131 - val_loss: 0.7365 - val_accuracy: 0.5145
Epoch 5/300
80/80 [==============================] - 4s 51ms/step - loss: 0.6912 - accuracy: 0.5134 - val_loss: 0.7562 - val_accuracy: 0.5014
Epoch 6/300
80/80 [==============================] - 4s 52ms/step - loss: 0.6900 - accuracy: 0.5197 - val_loss: 0.7591 - val_accuracy: 0.5016
Epoch 7/300
80/80 [==============================] - 4s 46ms/step - loss: 0.6893 - accuracy: 0.5264 - val_loss: 0.8331 - val_accuracy: 0.5030
Epoch 

80/80 [==============================] - 4s 47ms/step - loss: 0.6594 - accuracy: 0.5726 - val_loss: 3.3085 - val_accuracy: 0.4986
Epoch 59/300
80/80 [==============================] - 4s 47ms/step - loss: 0.6582 - accuracy: 0.5731 - val_loss: 3.5384 - val_accuracy: 0.4999
Epoch 60/300
80/80 [==============================] - 4s 46ms/step - loss: 0.6595 - accuracy: 0.5727 - val_loss: 3.5570 - val_accuracy: 0.4931
Epoch 61/300
80/80 [==============================] - 4s 46ms/step - loss: 0.6578 - accuracy: 0.5764 - val_loss: 3.7213 - val_accuracy: 0.4993
Epoch 62/300
80/80 [==============================] - 4s 46ms/step - loss: 0.6567 - accuracy: 0.5759 - val_loss: 3.6929 - val_accuracy: 0.4911
Epoch 63/300
80/80 [==============================] - 4s 46ms/step - loss: 0.6548 - accuracy: 0.5804 - val_loss: 4.1839 - val_accuracy: 0.4948
Epoch 64/300
80/80 [==============================] - 4s 49ms/step - loss: 0.6581 - accuracy: 0.5706 - val_loss: 3.8080 - val_accuracy: 0.4897
Epoch 65/300

80/80 [==============================] - 4s 48ms/step - loss: 0.6356 - accuracy: 0.5969 - val_loss: 5.8983 - val_accuracy: 0.4876
Epoch 116/300
80/80 [==============================] - 4s 48ms/step - loss: 0.6335 - accuracy: 0.6009 - val_loss: 5.8885 - val_accuracy: 0.4865
Epoch 117/300
80/80 [==============================] - 4s 47ms/step - loss: 0.6346 - accuracy: 0.5958 - val_loss: 5.7847 - val_accuracy: 0.4875
Epoch 118/300
80/80 [==============================] - 4s 55ms/step - loss: 0.6305 - accuracy: 0.6023 - val_loss: 6.1120 - val_accuracy: 0.4909
Epoch 119/300
80/80 [==============================] - 4s 48ms/step - loss: 0.6281 - accuracy: 0.6022 - val_loss: 6.3131 - val_accuracy: 0.4894
Epoch 120/300
80/80 [==============================] - 4s 47ms/step - loss: 0.6276 - accuracy: 0.6027 - val_loss: 6.2525 - val_accuracy: 0.4876
Epoch 121/300
80/80 [==============================] - 4s 47ms/step - loss: 0.6270 - accuracy: 0.6026 - val_loss: 6.4527 - val_accuracy: 0.4865
Epoch 

Epoch 172/300
80/80 [==============================] - 6s 79ms/step - loss: 0.6105 - accuracy: 0.6163 - val_loss: 9.1564 - val_accuracy: 0.4954
Epoch 173/300
80/80 [==============================] - 6s 79ms/step - loss: 0.6061 - accuracy: 0.6199 - val_loss: 9.0541 - val_accuracy: 0.4944
Epoch 174/300
80/80 [==============================] - 6s 78ms/step - loss: 0.6043 - accuracy: 0.6201 - val_loss: 8.6648 - val_accuracy: 0.4979
Epoch 175/300
80/80 [==============================] - 6s 78ms/step - loss: 0.6034 - accuracy: 0.6208 - val_loss: 8.7967 - val_accuracy: 0.4983
Epoch 176/300
80/80 [==============================] - 6s 78ms/step - loss: 0.6019 - accuracy: 0.6232 - val_loss: 8.9974 - val_accuracy: 0.4979
Epoch 177/300
80/80 [==============================] - 6s 78ms/step - loss: 0.6047 - accuracy: 0.6212 - val_loss: 9.4815 - val_accuracy: 0.4960
Epoch 178/300
80/80 [==============================] - 6s 78ms/step - loss: 0.6059 - accuracy: 0.6185 - val_loss: 9.7068 - val_accuracy:

80/80 [==============================] - 7s 82ms/step - loss: 0.5909 - accuracy: 0.6317 - val_loss: 14.8445 - val_accuracy: 0.5050
Epoch 229/300
80/80 [==============================] - 6s 78ms/step - loss: 0.5852 - accuracy: 0.6348 - val_loss: 15.5144 - val_accuracy: 0.5015
Epoch 230/300
80/80 [==============================] - 6s 78ms/step - loss: 0.5810 - accuracy: 0.6397 - val_loss: 15.6018 - val_accuracy: 0.4989
Epoch 231/300
80/80 [==============================] - 6s 78ms/step - loss: 0.5844 - accuracy: 0.6354 - val_loss: 15.8893 - val_accuracy: 0.5009
Epoch 232/300
80/80 [==============================] - 6s 78ms/step - loss: 0.5867 - accuracy: 0.6320 - val_loss: 15.1932 - val_accuracy: 0.4992
Epoch 233/300
80/80 [==============================] - 6s 78ms/step - loss: 0.5915 - accuracy: 0.6310 - val_loss: 15.0326 - val_accuracy: 0.5008
Epoch 234/300
80/80 [==============================] - 6s 79ms/step - loss: 0.5880 - accuracy: 0.6309 - val_loss: 12.9823 - val_accuracy: 0.5026

80/80 [==============================] - 4s 45ms/step - loss: 0.5635 - accuracy: 0.6504 - val_loss: 16.3008 - val_accuracy: 0.5064
Epoch 285/300
80/80 [==============================] - 4s 45ms/step - loss: 0.5652 - accuracy: 0.6526 - val_loss: 15.6851 - val_accuracy: 0.5047
Epoch 286/300
80/80 [==============================] - 4s 45ms/step - loss: 0.5671 - accuracy: 0.6497 - val_loss: 16.0965 - val_accuracy: 0.5046
Epoch 287/300
80/80 [==============================] - 4s 45ms/step - loss: 0.5634 - accuracy: 0.6500 - val_loss: 16.2953 - val_accuracy: 0.5054
Epoch 288/300
80/80 [==============================] - 4s 45ms/step - loss: 0.5627 - accuracy: 0.6523 - val_loss: 16.8704 - val_accuracy: 0.5054
Epoch 289/300
80/80 [==============================] - 4s 45ms/step - loss: 0.5592 - accuracy: 0.6550 - val_loss: 15.9773 - val_accuracy: 0.5048
Epoch 290/300
80/80 [==============================] - 4s 46ms/step - loss: 0.5659 - accuracy: 0.6493 - val_loss: 16.3268 - val_accuracy: 0.5056

In [14]:
X_dynamic_train_reshaped[:,-12:,:].mean(axis=0)[:,1]

array([0.00210181, 0.00325116, 0.00205564, 0.00144565, 0.00137958,
       0.00140342, 0.00069939, 0.00134703, 0.00125825, 0.00117195,
       0.00166388, 0.00045813])

In [15]:
y_train

SK_ID_CURR
100002    1
100010    0
100017    0
100021    0
100030    0
         ..
337787    1
337794    1
337816    1
337820    1
337825    0
Name: TARGET, Length: 33265, dtype: int64

In [16]:
loss_train = history.history['auc']
loss_val = history.history['val_auc']
epochs = range(300)
plt.plot(epochs, loss_train, 'g', label='Training acc')
plt.plot(epochs, loss_val, 'b', label='validation acc')
plt.title('Training and Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

KeyError: 'auc'

In [ ]:

loss_train = history.history['auc']
loss_val = history.history['val_auc']
epochs = range(500)
plt.plot(epochs, loss_train, 'g', label='Training acc')
plt.plot(epochs, loss_val, 'b', label='validation acc')
plt.title('Training and Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

In [ ]:
loss_train = history.history['auc_9']
loss_val = history.history['val_auc_9']
epochs = range(300)
plt.plot(epochs, loss_train, 'g', label='Training acc')
plt.plot(epochs, loss_val, 'b', label='validation acc')
plt.title('Training and Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

In [ ]:

loss_train = history.history['auc']
loss_val = history.history['val_auc']
epochs = range(500)
plt.plot(epochs, loss_train, 'g', label='Training acc')
plt.plot(epochs, loss_val, 'b', label='validation acc')
plt.title('Training and Validation acc')
plt.xlabel('Epochs')
plt.ylabel('Acc')
plt.legend()
plt.show()

In [ ]:
from sklearn.metrics import roc_curve
y_pred = model1.predict([X_dynamic_test, X_static_test]).ravel()
fpr_keras, tpr_keras, thresholds_keras = roc_curve(y_test, y_pred)

In [ ]:
from sklearn.metrics import auc
auc_keras = auc(fpr_keras, tpr_keras)

In [ ]:
plt.figure(1)
plt.plot([0, 1], [0, 1], 'k--')
plt.plot(fpr_keras, tpr_keras, label='Keras (area = {:.3f})'.format(auc_keras))
# plt.plot(fpr_rf, tpr_rf, label='RF (area = {:.3f})'.format(auc_rf))
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
print(thresholds_keras)